In [43]:
import numpy as np

import csv
from tqdm import tqdm

import matplotlib
import matplotlib.pyplot as plt
import scienceplots
plt.style.use(['science'])
matplotlib.rcParams["font.size"] = "12"

import torch
import torch.nn as nn
from torch.autograd import grad

from scipy.integrate import solve_ivp

In [44]:
class FeedForwardNetwork(nn.Module):
    def __init__(self, hidden_layers, hidden_dim, input_dim=1, output_dim=1, init_rule=torch.nn.init.uniform_):
        super(FeedForwardNetwork, self).__init__()
        
        self.L = hidden_layers
        self.W = hidden_dim
        
        self.model = nn.Sequential()
        self.activation = nn.Tanh()
        
        inp_linear = nn.Linear(input_dim, hidden_dim)
        init_rule(inp_linear.weight)
        out_linear = nn.Linear(hidden_dim, output_dim)
        init_rule(out_linear.weight)
        
        self.model.add_module('input', inp_linear)
        self.model.add_module('activ0', self.activation)
        for i in range(hidden_layers - 1):
            linear = nn.Linear(hidden_dim, hidden_dim)
            init_rule(linear.weight)
            self.model.add_module(f'linear{i+1}', linear)
            self.model.add_module(f'activ{i+1}', self.activation)
        self.model.add_module('output', out_linear)
        
    def forward(self, x):
        return self.model(x)
    
    def weights_norm(self):
        norms = []
        with torch.no_grad():
            for name, parameter in self.model.named_parameters():
                norms.append(torch.linalg.norm(parameter))
        return np.array(norms)
    
    def init_weights(self, initialization):
        for param in self.model.parameters():
            if len(param.shape) > 1:
                initialization(param)
                
def rmse(predicts, target):
    '''
    Note that for diffusion both arrays have shape (T, X).
    '''
    return np.sqrt(np.square(predicts - target).mean())

---
## Lotka-Volterra Problem

In [45]:
def plot_lotkavolterra(
    t, predictions, numerical, title=None, figsize=(5, 5), 
    show=True, save=False, path='./plot', dpi=300
    ):
    
    fig = plt.figure(figsize=figsize)
    plt.plot(t, predictions[0], label=r'$\mathcal{X}(t)$')
    plt.plot(t, predictions[1], label=r'$\mathcal{Y}(t)$')
    plt.plot(t, numerical[0], label=r'$x(t)$')
    plt.plot(t, numerical[1], label=r'$y(t)$')
    
    plt.xlabel('t')
    plt.ylabel('Population size')
    
    plt.legend()
    plt.title(title)
    
    if save:
        plt.savefig(f'{path}.png', dpi=dpi)
    
    if show:
        plt.show()
    else:
        plt.close()

In [46]:
class LotkaVolterraProblem:
    def __init__(self, T, params, initial_conditions):
        
        self.T = T
        self.alpha, self.beta, self.delta, self.gamma = params
        self.init_vals = torch.tensor(initial_conditions)

        self.numerical_solution = self._solve()
    
    def loss_initial(self, model):
        zero = torch.tensor([0.], requires_grad=True)
        x = model(zero)
        return torch.mean(torch.square(x - self.init_vals))
    
    def loss_physical(self, model, t):
        xy = model(t)
        x = xy[:,[0]]
        y = xy[:,[1]]

        dX = grad(x, t, grad_outputs=torch.ones_like(x), create_graph=True)[0]
        dY = grad(y, t, grad_outputs=torch.ones_like(y), create_graph=True)[0]
        
        loss_dX = torch.mean(torch.square(dX - self.alpha * x + self.beta * x * y))
        loss_dY = torch.mean(torch.square(dY - self.delta * x * y + self.gamma * y))
        
        return loss_dX, loss_dY
    
    def _solve(self):
        def lotka_volterra(t, y, alpha, beta, delta, gamma):
            x, y = y
            dx_dt = alpha * x - beta * x * y
            dy_dt = delta * x * y - gamma * y
            return [dx_dt, dy_dt]

        solution = solve_ivp(lotka_volterra, 
                             (0, self.T),
                             self.init_vals, 
                             method='RK45',
                             args=(self.alpha, self.beta, self.delta, self.gamma), 
                             t_eval=np.linspace(0, self.T, 128))
        
        return solution.y

In [47]:
def LotkaVolterraTraining(
    problem,
    model,
    w1, w2, w3, num_iters, N_D, lr,
    print_every=0, collect_every=0, save_every=0
):
    test_points = torch.linspace(0, problem.T, 128).reshape(-1, 1)
    t = torch.linspace(0, problem.T, N_D, requires_grad=True).reshape(-1, 1)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    losses, errors = [], []

    for i in range(num_iters + 1):

        optimizer.zero_grad()

        L_I = problem.loss_initial(model)
        L_X, L_Y = problem.loss_physical(model, t)

        L = w1 * L_I + w2 * L_X + w3 * L_Y

        L.backward()
        optimizer.step()
        
        if collect_every > 0 and i % collect_every == 0:
            preds = model(test_points).detach().numpy()
            x = preds[:,0].flatten()
            y = preds[:,1].flatten()

            error = (rmse(x, problem.numerical_solution[0]) + rmse(y, problem.numerical_solution[1])) / 2

            losses.append(np.array([L_I.item(), L_X.item(), L_Y.item(), L.  item()]))
            errors.append(error)
        
        if print_every > 0 and i % print_every == 0:
            print(f'Iteration {i} --- RMSE {error}')
                
        if save_every > 0 and i % save_every == 0:
            plot_lotkavolterra(
            test_points, [x, y], problem.numerical_solution, title=f'Iteration {i}', figsize=(6, 4), show=False, save=True, path=f'./training/lotkavolterra/Iteration {i}'
        )
    
    return np.array(losses), np.array(errors)

---
## Diffusion Problem

In [48]:
def plot_diffusion(x, t, predictions, size=(10, 5), show=True, save=False, path='./plot', dpi=300):
    X, T = np.meshgrid(x, t)
    fig = plt.figure(figsize=size)

    ax = fig.add_subplot(1, 2, 1, projection='3d')
    ax.plot_surface(X, T, predictions, cmap='viridis')
    ax.set_xlabel('x')
    ax.set_ylabel('t')

    ax = fig.add_subplot(1, 2, 2)
    ax.imshow(predictions, origin='lower', aspect='auto', cmap='viridis', 
              extent=[x.min(), x.max(), t.min(), t.max()])
    ax.set_xlabel('x')
    ax.set_ylabel('t')

    plt.tight_layout()
    
    if save:
        plt.savefig(f'{path}.png', dpi=dpi)
    
    if show:
        plt.show()
    else:
        plt.close()

In [49]:
class DiffusionProblem:
    def __init__(self, D, boundaries, resolution, b_values):
        self.D = D
        self.L, self.R, self.T = boundaries
        self.left_boundary, self.right_boundary, self.initial_values = b_values
        
        self.Nt, self.Nx = resolution
        self.numerical_solution = self._solve()
        self.t = torch.linspace(0, self.T, self.Nt)
        self.x = torch.linspace(self.L, self.R, self.Nx)
    
    def loss_boundary(self, model, t_left, t_right, x_init):
        
        def nearest_index(array, values):
            values = [np.abs(array - v).argmin() for v in values]
            return values
        
        left_values =  self.left_boundary[nearest_index(self.t, t_left)  ]
        right_values = self.right_boundary[nearest_index(self.t, t_right)]
        init_values =  self.initial_values[nearest_index(self.x, x_init) ]
        
        left =  model(torch.vstack([torch.ones_like(t_left)  * self.L, t_left]).T)
        right = model(torch.vstack([torch.ones_like(t_right) * self.R, t_right]).T)
        init =  model(torch.vstack([x_init, torch.zeros_like(x_init)]).T)
        
        left_error = torch.square(left_values - left.flatten()).mean()
        right_error = torch.square(right_values - right.flatten()).mean()
        init_error = torch.square(init_values - init.flatten()).mean()
        
        return left_error + right_error, init_error
    
    def loss_physical(self, model, x, t):
        u = model(torch.hstack([x, t]))
        
        ut =  grad(u,  t, grad_outputs=torch.ones_like(t), create_graph=True)[0]
        ux =  grad(u,  x, grad_outputs=torch.ones_like(x), create_graph=True)[0]
        uxx = grad(ux, x, grad_outputs=torch.ones_like(x), create_graph=True)[0]
    
        return torch.mean(torch.square(ut - self.D * uxx))
        
    def _solve(self):
        Nt, Nx = self.Nt, self.Nx
        
        dx = (self.R - self.L) / Nx     # Spatial step size
        dt = self.T / Nt                # Time step size
        alpha = self.D * dt / (2 * dx ** 2)

        # Crank-Nicholson method
        u = np.zeros((Nt, Nx))

        # Initial condition
        u[0, :] = self.initial_values

        # Boundary conditions
        u[:, 0], u[:, -1] = self.left_boundary, self.right_boundary

        A = np.diag((1 + 2*alpha) * np.ones(Nx)) + np.diag(-alpha * np.ones(Nx-1), 1) + np.diag(-alpha * np.ones(Nx-1), -1)
        B = np.diag((1 - 2*alpha) * np.ones(Nx)) + np.diag(alpha * np.ones(Nx-1), 1) + np.diag(alpha * np.ones(Nx-1), -1)
            
        A_reversed = np.linalg.inv(A)

        for n in range(0, Nt - 1):
            b = np.dot(B, u[n, :])
            u[n+1, :] = A_reversed @ b

        return u

In [50]:
def train_diffusion(
    problem,
    model,
    N_LB, N_RB, N_I, N_D,
    coef, lr, num_iters,
    print_every=0, collect_every=0, save_every=0
):

    x_sampled = torch.tensor(np.random.uniform(problem.L, problem.R, N_I), dtype=torch.float32)
    t_left_sampled =  torch.tensor(np.random.uniform(0, problem.T, N_LB), dtype=torch.float32)
    t_right_sampled = torch.tensor(np.random.uniform(0, problem.T, N_RB), dtype=torch.float32)

    x_pts = torch.tensor(np.random.uniform(problem.L, problem.R, N_D), requires_grad=True, dtype=torch.float32).reshape(-1, 1)
    t_pts = torch.tensor(np.random.uniform(0, problem.T, N_D), requires_grad=True, dtype=torch.float32).reshape(-1, 1)

    test_points = torch.cartesian_prod(problem.x, problem.t)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    losses, errors = [], []

    for i in range(num_iters + 1):

        optimizer.zero_grad()

        L_B, L_I = problem.loss_boundary(model, t_left_sampled, t_right_sampled, x_sampled)
        L_D = problem.loss_physical(model, x_pts, t_pts)

        total = coef * (L_B + L_I) + (1 - coef) * L_D

        total.backward()
        optimizer.step()
                
        if collect_every > 0 and i % collect_every == 0:
            preds = model(test_points).reshape(problem.Nx, problem.Nt).detach().numpy()
                
            error = rmse(preds.T, problem.numerical_solution)
                
            losses.append(np.array([L_I.item(), L_B.item(), L_D.item(), total.item()]))
            errors.append(error)
                
        if print_every > 0 and i % print_every == 0:
            print(f'Iteration {i} --- RMSE {error}')
                
        if save_every > 0 and i % save_every == 0:
            plot_diffusion(problem.x, problem.t, preds.T, show=False, save=True, path=f'./training/diffusion/Iteration {i}')
                
    return np.array(losses), np.array(errors)

---
## Initialization Methods

In [53]:
init_methods = [
    (torch.nn.init.eye_, 'Eye'),
    (torch.nn.init.zeros_, 'Zeros'),
    (torch.nn.init.normal_, 'Normal'),
    (torch.nn.init.uniform_, 'Uniform'),
    (torch.nn.init.orthogonal_, 'Orthogonal'),
    (torch.nn.init.kaiming_normal_, 'Kaiming Normal'),
    (torch.nn.init.kaiming_uniform_, 'Kaiming Uniform'),
]

num_trainings = 15

In [ ]:
# problem = LotkaVolterraProblem(
#     25., (0.4, 0.1, 0.1, 0.6), [10., 5.]
#     )

# w1, w2, w3 = 1, 2, 2
# num_iters = 20000
# N_D = 1024
# lr = 1e-3

In [ ]:
# D = 0.5
# L, R, T = 0, 1, 0.5
# Nt, Nx = 1000, 250
# left_boundary = right_boundary = torch.zeros(Nt)
# initial_conditions = torch.sin(2 * np.pi * torch.linspace(L, R, Nx)) ** 2

# problem = DiffusionProblem(
#     D, (L, R, T), (Nt, Nx),
#     (left_boundary, right_boundary, initial_conditions)
# )

# N_I, N_LB, N_RB, N_D = 128, 64, 64, 2048
# coef = 0.8
# lr = 1e-3
# num_iters = 7500

In [81]:
import pandas as pd

def get_report(n_rows, lv_threshold, diff_threshold):

    lv = pd.read_csv('./lotkavolterra_initializations.csv', names=['Method', 'RMSE'])
    lv['Task'] = ['Lotka-Volterra' for _ in range(n_rows)]
    lv['Class'] = lv['RMSE'] < lv_threshold
    
    diff = pd.read_csv('./diffusion_initializations.csv', names=['Method', 'RMSE'])
    diff['Task'] = ['Diffusion' for _ in range(n_rows)]
    diff['Class'] = diff['RMSE'] < diff_threshold
    
    return pd.concat([lv, diff])

In [82]:
report = get_report(
    len(init_methods) * num_trainings,
    lv_threshold=0.2, diff_threshold=0.01
    )

In [94]:
grouped = report.groupby(['Task', 'Class', 'Method'])

In [95]:
grouped.describe()

RMSE                               \
                                     count      mean       std      min   
Task           Class Method                                               
Diffusion      False Eye               8.0  0.013830  0.002247  0.01016   
                     Kaiming Normal    4.0  0.011967  0.002275  0.01061   
                     Kaiming Uniform   3.0  0.011747  0.001779  0.01067   
                     Normal           15.0  0.036533  0.005249  0.02553   
                     Ones             15.0  0.182225  0.102934  0.14910   
                     Orthogonal       11.0  0.014235  0.004693  0.01004   
                     Uniform          15.0  0.072469  0.028925  0.04633   
                     Xavier Normal    13.0  0.016355  0.004651  0.01057   
                     Xavier Uniform   13.0  0.018112  0.006489  0.01085   
                     Zeros            15.0  0.035585  0.010856  0.01420   
               True  Eye               7.0  0.007483  0.000934  0.00631   
                     Kaiming Normal   11.0  0.007046  0.001555  0.00517   
                     Kaiming Uniform  12.0  0.008048  0.001216  0.00561   
                     Orthogonal        4.0  0.007618  0.002155  0.00494   
                     Xavier Normal     2.0  0.008355  0.000530  0.00798   
                     Xavier Uniform    2.0  0.008250  0.001089  0.00748   
Lotka-Volterra False Eye               9.0  3.473254  1.813184  0.25713   
                     Kaiming Normal   15.0  3.909422  1.416706  0.29652   
                     Kaiming Uniform  15.0  3.917881  1.433695  0.38375   
                     Normal           13.0  2.518866  2.102591  0.20056   
                     Ones             15.0  2.155833  0.354404  1.83784   
                     Orthogonal       15.0  4.386090  0.000791  4.38378   
                     Uniform          15.0  1.589525  1.184310  0.58618   
                     Xavier Normal    13.0  4.386182  0.000606  4.38479   
                     Xavier Uniform   14.0  4.386829  0.002153  4.38117   
                     Zeros             7.0  0.317960  0.051653  0.24831   
               True  Eye               6.0  0.126517  0.053698  0.03155   
                     Normal            2.0  0.075320  0.017112  0.06322   
                     Xavier Normal     2.0  0.132110  0.040418  0.10353   
                     Xavier Uniform    1.0  0.087930       NaN  0.08793   
                     Zeros             8.0  0.040315  0.024056  0.02340   

                                                                             
                                           25%       50%       75%      max  
Task           Class Method                                                  
Diffusion      False Eye              0.013125  0.013755  0.014780  0.01703  
                     Kaiming Normal   0.010797  0.010945  0.012115  0.01537  
                     Kaiming Uniform  0.010720  0.010770  0.012285  0.01380  
                     Normal           0.033435  0.035870  0.040985  0.04328  
                     Ones             0.149110  0.149220  0.151880  0.54526  
                     Orthogonal       0.010645  0.012840  0.016975  0.02546  
                     Uniform          0.059695  0.061700  0.071820  0.14884  
                     Xavier Normal    0.012490  0.015300  0.021470  0.02360  
                     Xavier Uniform   0.013870  0.015370  0.021860  0.03156  
                     Zeros            0.029305  0.037370  0.044005  0.05226  
               True  Eye              0.007040  0.007270  0.007780  0.00916  
                     Kaiming Normal   0.005740  0.006670  0.008625  0.00937  
                     Kaiming Uniform  0.007310  0.008040  0.008960  0.00960  
                     Orthogonal       0.006410  0.007860  0.009067  0.00981  
                     Xavier Normal    0.008167  0.008355  0.008542  0.00873  
                     Xavier Uniform   0.007865  0.008250  0.008635  0.00902  
Lotka

In [103]:
def percent(a):
    return a / 15 * 100

percent(12)

80.0